In [2]:
#申請token
import requests

# 替換成你自己的 Client ID 和 Client Secret
client_id = '***'
client_secret = '***'

# TDX 的 Token 取得網址
url = 'https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token'

# 要傳送的資料
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

# 發送 POST 請求
response = requests.post(url, data=data)

# 顯示回應結果
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("取得 Access Token 成功！")
    print("Access Token:", access_token)
else:
    print("取得 Access Token 失敗")
    print("狀態碼:", response.status_code)
    print("回應內容:", response.text)


In [ ]:
#呼叫單次資料
import requests

url = "***"

headers = {
    "Authorization": "Bearer ***"
}

response = requests.get(url, headers=headers)

print(response.status_code)     # 印出 200 表示成功
print(response.json())          # 印出 API 傳回來的資料


In [ ]:
#單次資料excel檔
import requests
import pandas as pd

# API URL 與 headers
url = "***"
headers = {
    "Authorization": "Bearer ***"
}

# 發送 GET 請求
response = requests.get(url, headers=headers)

# 確認是否成功取得資料
if response.status_code == 200:
    data = response.json()  # 取得回傳的 JSON 資料
    
    # 將整個 JSON 資料轉換為 DataFrame
    df = pd.json_normalize(data)  # 這樣會將 JSON 中的所有資料都展開成表格
    
    # 儲存為 Excel 檔案
    df.to_excel('檔名.xlsx', index=False, engine='openpyxl')
    print("資料已成功儲存為 Excel 檔案！")
else:
    print(f"請求失敗，錯誤代碼：{response.status_code}")

In [ ]:
#第一次建資料庫+迴圈
import requests
import sqlite3
import time

# 👉 請把以下內容換成妳自己的 access_token 和 API URL
ACCESS_TOKEN = "***"

URLS = {
    "A1": "https://tdx.transportdata.tw/api/basic/v2/Bus/RealTimeByFrequency/City/Taipei/和平幹線?%24select=PlateNumb%2CRouteUID%2CSubRouteUID%2CDirection%2CSpeed%2CGPSTime%2CUpdateTime%2CBusPosition&%24format=JSON",
    "A2": "https://tdx.transportdata.tw/api/basic/v2/Bus/RealTimeNearStop/City/Taipei/和平幹線?%24select=PlateNumb%2CRouteUID%2CSubRouteUID%2CDirection%2CStopUID%2CStopSequence%2CGPSTime%2CUpdateTime%2CStopName&%24format=JSON",
    "Estimate": "https://tdx.transportdata.tw/api/basic/v2/Bus/EstimatedTimeOfArrival/City/Taipei/和平幹線?%24select=StopUID%2CRouteUID%2CDirection%2CEstimateTime%2CUpdateTime%2CStopName&%24format=JSON"
}

HEADERS = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

# 建立 SQLite 資料庫與表格
def init_db():
    conn = sqlite3.connect("檔名.db")
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bus_realtime_position (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            PlateNumb TEXT, RouteUID TEXT, SubRouteUID TEXT, Direction INTEGER,
            Speed REAL, Azimuth INTEGER, DutyStatus INTEGER, BusStatus INTEGER,
            GPSTime TEXT, SrcUpdateTime TEXT, UpdateTime TEXT,
            PositionLon REAL, PositionLat REAL, GeoHash TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bus_stop_event (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            PlateNumb TEXT, RouteUID TEXT, SubRouteUID TEXT, Direction INTEGER,
            StopUID TEXT, StopSequence INTEGER, DutyStatus INTEGER, BusStatus INTEGER,
            A2EventType INTEGER, GPSTime TEXT, TripStartTimeType INTEGER,
            SrcUpdateTime TEXT, UpdateTime TEXT,
            StopName_Zh_tw TEXT, StopName_En TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bus_estimate_time (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            StopUID TEXT, RouteUID TEXT, Direction INTEGER, EstimateTime INTEGER,
            StopStatus INTEGER, SrcUpdateTime TEXT, UpdateTime TEXT,
            StopName_Zh_tw TEXT, StopName_En TEXT
        )
    ''')

    conn.commit()
    conn.close()

# 資料抓取與儲存
def fetch_and_save():
    conn = sqlite3.connect("檔名.db")
    cursor = conn.cursor()

    for i in range(240):  # 總共抓幾次，120次是兩小時
        print(f"\n🔄 第 {i+1} 次抓取中...")

        for key, url in URLS.items():
            try:
                response = requests.get(url, headers=HEADERS)
                response.raise_for_status()
                data = response.json()

                for item in data:
                    if key == "A1":
                        pos = item.get("BusPosition", {})
                        cursor.execute('''
                            INSERT INTO bus_realtime_position (
                                PlateNumb, RouteUID, SubRouteUID, Direction, Speed,
                                Azimuth, DutyStatus, BusStatus, GPSTime,
                                SrcUpdateTime, UpdateTime,
                                PositionLon, PositionLat, GeoHash
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                            item.get("PlateNumb"),
                            item.get("RouteUID"),
                            item.get("SubRouteUID"),
                            item.get("Direction"),
                            item.get("Speed"),
                            item.get("Azimuth"),
                            item.get("DutyStatus"),
                            item.get("BusStatus"),
                            item.get("GPSTime"),
                            item.get("SrcUpdateTime"),
                            item.get("UpdateTime"),
                            pos.get("PositionLon"),
                            pos.get("PositionLat"),
                            pos.get("GeoHash")
                        ))

                    elif key == "A2":
                        stop_name = item.get("StopName", {})
                        cursor.execute('''
                            INSERT INTO bus_stop_event (
                                PlateNumb, RouteUID, SubRouteUID, Direction,
                                StopUID, StopSequence, DutyStatus, BusStatus,
                                A2EventType, GPSTime, TripStartTimeType,
                                SrcUpdateTime, UpdateTime,
                                StopName_Zh_tw, StopName_En
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                            item.get("PlateNumb"),
                            item.get("RouteUID"),
                            item.get("SubRouteUID"),
                            item.get("Direction"),
                            item.get("StopUID"),
                            item.get("StopSequence"),
                            item.get("DutyStatus"),
                            item.get("BusStatus"),
                            item.get("A2EventType"),
                            item.get("GPSTime"),
                            item.get("TripStartTimeType"),
                            item.get("SrcUpdateTime"),
                            item.get("UpdateTime"),
                            stop_name.get("Zh_tw"),
                            stop_name.get("En")
                        ))

                    elif key == "Estimate":
                        stop_name = item.get("StopName", {})
                        cursor.execute('''
                            INSERT INTO bus_estimate_time (
                                StopUID, RouteUID, Direction, EstimateTime,
                                StopStatus, SrcUpdateTime, UpdateTime,
                                StopName_Zh_tw, StopName_En
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                            item.get("StopUID"),
                            item.get("RouteUID"),
                            item.get("Direction"),
                            item.get("EstimateTime"),
                            item.get("StopStatus"),
                            item.get("SrcUpdateTime"),
                            item.get("UpdateTime"),
                            stop_name.get("Zh_tw"),
                            stop_name.get("En")
                        ))

                conn.commit()
                print(f"✅ {key} 寫入完成（{len(data)} 筆）")

            except Exception as e:
                print(f"❌ {key} 發生錯誤：{e}")

        time.sleep(30)  # 每分鐘抓一次

    conn.close()
    print("\n🎉 抓取與儲存完成")

# 主程式
if __name__ == "__main__":
    init_db()
    fetch_and_save()


In [ ]:
#第二次跑迴圈插入資料
import requests
import sqlite3
import time
ACCESS_TOKEN = "***"

URLS = {
    "A1": "https://tdx.transportdata.tw/api/basic/v2/Bus/RealTimeByFrequency/City/Taipei/18?%24select=PlateNumb%2CRouteUID%2CSubRouteUID%2CDirection%2CSpeed%2CGPSTime%2CUpdateTime%2CBusPosition&%24format=JSON",
    "A2": "https://tdx.transportdata.tw/api/basic/v2/Bus/RealTimeNearStop/City/Taipei/18?%24select=PlateNumb%2CRouteUID%2CSubRouteUID%2CDirection%2CStopUID%2CStopSequence%2CGPSTime%2CUpdateTime%2CStopName&%24format=JSON",
    "Estimate": "https://tdx.transportdata.tw/api/basic/v2/Bus/EstimatedTimeOfArrival/City/Taipei/18?%24select=StopUID%2CRouteUID%2CDirection%2CEstimateTime%2CUpdateTime%2CStopName&%24format=JSON"
}

HEADERS = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

def fetch_and_save():
    conn = sqlite3.connect("檔名.db")
    cursor = conn.cursor()

    for i in range(240):  # 再跑兩小時
        print(f"\n⏱️ 第 {i+1} 次抓取中...")

        for key, url in URLS.items():
            try:
                response = requests.get(url, headers=HEADERS)
                response.raise_for_status()
                data = response.json()

                for item in data:
                    if key == "A1":
                        pos = item.get("BusPosition", {})
                        cursor.execute('''
                            INSERT INTO bus_realtime_position (
                                PlateNumb, RouteUID, SubRouteUID, Direction, Speed,
                                Azimuth, DutyStatus, BusStatus, GPSTime,
                                SrcUpdateTime, UpdateTime,
                                PositionLon, PositionLat, GeoHash
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                            item.get("PlateNumb"),
                            item.get("RouteUID"),
                            item.get("SubRouteUID"),
                            item.get("Direction"),
                            item.get("Speed"),
                            item.get("Azimuth"),
                            item.get("DutyStatus"),
                            item.get("BusStatus"),
                            item.get("GPSTime"),
                            item.get("SrcUpdateTime"),
                            item.get("UpdateTime"),
                            pos.get("PositionLon"),
                            pos.get("PositionLat"),
                            pos.get("GeoHash")
                        ))

                    elif key == "A2":
                        stop_name = item.get("StopName", {})
                        cursor.execute('''
                            INSERT INTO bus_stop_event (
                                PlateNumb, RouteUID, SubRouteUID, Direction,
                                StopUID, StopSequence, DutyStatus, BusStatus,
                                A2EventType, GPSTime, TripStartTimeType,
                                SrcUpdateTime, UpdateTime,
                                StopName_Zh_tw, StopName_En
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                            item.get("PlateNumb"),
                            item.get("RouteUID"),
                            item.get("SubRouteUID"),
                            item.get("Direction"),
                            item.get("StopUID"),
                            item.get("StopSequence"),
                            item.get("DutyStatus"),
                            item.get("BusStatus"),
                            item.get("A2EventType"),
                            item.get("GPSTime"),
                            item.get("TripStartTimeType"),
                            item.get("SrcUpdateTime"),
                            item.get("UpdateTime"),
                            stop_name.get("Zh_tw"),
                            stop_name.get("En")
                        ))

                    elif key == "Estimate":
                        stop_name = item.get("StopName", {})
                        cursor.execute('''
                            INSERT INTO bus_estimate_time (
                                StopUID, RouteUID, Direction, EstimateTime,
                                StopStatus, SrcUpdateTime, UpdateTime,
                                StopName_Zh_tw, StopName_En
                            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                            item.get("StopUID"),
                            item.get("RouteUID"),
                            item.get("Direction"),
                            item.get("EstimateTime"),
                            item.get("StopStatus"),
                            item.get("SrcUpdateTime"),
                            item.get("UpdateTime"),
                            stop_name.get("Zh_tw"),
                            stop_name.get("En")
                        ))

                conn.commit()
                print(f"✅ {key} 寫入成功（{len(data)} 筆）")

            except Exception as e:
                print(f"❌ {key} 發生錯誤：{e}")

        time.sleep(60)

    conn.close()
    print("\n🎉 資料收集結束")

# 執行觀察
if __name__ == "__main__":
    fetch_and_save()


In [ ]:
#查看前幾筆資料
import sqlite3
import pandas as pd

conn = sqlite3.connect("檔名.db")
df = pd.read_sql_query("SELECT * FROM EstimatedTime LIMIT 10", conn)
print(df)
conn.close()